In [118]:
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score
from sklearn.preprocessing import LabelEncoder
from datasets import Dataset
import pandas as pd
from tqdm.notebook import tqdm
import ahocorasick

In [119]:
adv = pd.read_csv("subori.csv")

adv.sentence = adv.apply(lambda x: x["sentence"].replace(x['adverb_text'], f"<mark>{x['adverb_text']}</mark>"), axis=1)

In [120]:
adv

,sentence,adverb_type,adverb_position,verb_class,sentence_mood,adverb_scope,comma_intonation,adverb_text,adverb_count,subject_animacy,negation_scope,lexical_adverb_category
0,Prepare for an honest conversation with your k...,manner,not-modifying-verb,no_head_verb,imperative,unknown,False,honestly,1,inanimate,no_negation,evaluative
1,But what <mark>exactly</mark> sparked the rise...,context-free,not-modifying-verb,no_head_verb,interrogative,unknown,False,exactly,1,inanimate,no_negation,other
2,"<mark>Clearly</mark>, Larry is a power-hungry ...",context-free,not-modifying-verb,no_head_verb,declarative,S,True,Clearly,1,animate,no_negation,modal
3,"<mark>Unfortunately</mark>, it isnt a simple a...",context-free,not-modifying-verb,no_head_verb,declarative,S,True,Unfortunately,1,inanimate,adv_before_neg,evaluative
4,"<mark>Unfortunately</mark>, it didn’t work.",context-free,not-modifying-verb,no_head_verb,declarative,unknown,True,Unfortunately,1,inanimate,no_negation,evaluative
...,...,...,...,...,...,...,...,...,...,...,...,...
251,"“(Gilmore, Smith 31) God needs his followers t...",manner,imediate-pre-verbal,other,declarative,VP,False,obediently,1,inanimate,no_negation,other
252,The woman in blue listens to the discussion ta...,manner,imediate-pos-verbal,other,declarative,VP,False,passively,1,inanimate,no_negation,other
253,They did not expect to have an opening in the ...,context-free,pre-verbal,motion,declarative,VP,False,so,2,animate,neg_before_adv,degree
254,"At a book reading he gave in Washington, sever...",manner,imediate-pre-verbal,other,declarative,S,False,angrily,1,animate,no_negation,other


In [121]:
texts = adv.sentence.values.tolist()
labels = [int(label == 'subject-oriented') for label in adv.adverb_type.values]

In [122]:
# Train/test split
train_texts, val_texts, train_labels, val_labels = train_test_split(texts, labels, test_size=0.2, stratify=labels)

In [123]:
num_labels = 2

In [134]:
# Tokenizer and model
model_name = "answerdotai/ModernBERT-base"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=num_labels)

Some weights of ModernBertForSequenceClassification were not initialized from the model checkpoint at answerdotai/ModernBERT-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [135]:
# Tokenize
def tokenize_fn(texts):
    return tokenizer(texts, padding=True, truncation=True, return_tensors="pt")

In [136]:
train_encodings = tokenize_fn(train_texts)
val_encodings = tokenize_fn(val_texts)

In [137]:
# Create Hugging Face Datasets
train_dataset = Dataset.from_dict({
    'input_ids': train_encodings['input_ids'],
    'attention_mask': train_encodings['attention_mask'],
    'labels': torch.tensor(train_labels)
})
val_dataset = Dataset.from_dict({
    'input_ids': val_encodings['input_ids'],
    'attention_mask': val_encodings['attention_mask'],
    'labels': torch.tensor(val_labels)
})

In [138]:
# Metric
def compute_metrics(pred):
    preds = pred.predictions.argmax(-1)
    labels = pred.label_ids
    return {"accuracy": accuracy_score(labels, preds), "f1": f1_score(labels, preds, average="weighted")}

In [140]:
# Training config
training_args = TrainingArguments(
    output_dir="./results",
    eval_strategy="epoch",
    save_strategy="epoch",
    num_train_epochs=5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    metric_for_best_model="f1",
    logging_dir="./logs",
    load_best_model_at_end=True,
)

In [141]:
# Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics,
)

In [142]:
# Train and evaluate
trainer.train()
trainer.evaluate()

Epoch,Training Loss,Validation Loss,Accuracy,F1
1,No log,0.369779,0.846154,0.775641
2,No log,0.335372,0.846154,0.804910
3,No log,0.625673,0.788462,0.807273
4,No log,0.703553,0.846154,0.836874
5,No log,0.803668,0.846154,0.804910


{'eval_loss': 0.7035534381866455,
 'eval_accuracy': 0.8461538461538461,
 'eval_f1': 0.8368742368742369,
 'eval_runtime': 8.3651,
 'eval_samples_per_second': 6.216,
 'eval_steps_per_second': 0.837,
 'epoch': 5.0}

In [143]:
def tokenize_function(example):
    return tokenizer(example["sentence"], truncation=True, padding=True)

In [108]:
import spacy

In [109]:
nlp = spacy.load("en_core_web_sm")

In [110]:
def get_adverb_text(doc):
    for token in doc:
        if token.pos_ == "ADV":
            return token.text

In [111]:
def grammatical_features(examples):
    doc = nlp(examples['sentence'])
    return {
        "adverb_text": get_adverb_text(doc),
    }

In [112]:
words = ahocorasick.Automaton()

with open("subject-oriented-lexicons.txt") as fin:
    for idx, line in enumerate(fin.readlines()):
        line = line.strip()
        words.add_word(line, idx)
        
words.make_automaton()

In [115]:
# Load the Parquet file back into a Hugging Face Dataset, sep="\t"
data = Dataset.from_parquet("all_adverbs.parquet").filter(lambda x: sum([1 for end_index, val in words.iter(x["sentence"].lower())]) == 1)
data = data.map(grammatical_features).filter(lambda x: x["adverb_text"] is not None).map(lambda x: {"sentence": x["sentence"].replace(x["adverb_text"], f"<mark>{x['adverb_text']}</mark>")}).filter(lambda x: x["sentence"] not in adv.sentence)

Loading dataset shards:   0%|          | 0/25 [00:00<?, ?it/s]

Filter:   0%|          | 0/56830279 [00:00<?, ? examples/s]

Map:   0%|          | 0/397814 [00:00<?, ? examples/s]

Filter:   0%|          | 0/397814 [00:00<?, ? examples/s]

Map:   0%|          | 0/395585 [00:00<?, ? examples/s]

Filter:   0%|          | 0/395585 [00:00<?, ? examples/s]

In [145]:
data.to_csv("subject-oriented_ALL.csv")
data = data.to_pandas()

Creating CSV from Arrow format:   0%|          | 0/396 [00:00<?, ?ba/s]

In [152]:
data.sentence = data.sentence.str.replace("\n", "").replace("\r", "")

In [159]:
data[data.adverb_text.str.isalpha()].groupby("adverb_text").first().reset_index().sample(frac=1.)[["sentence", "adverb_text"]].to_csv("pipipi.csv", index=False)

In [144]:
predict_dataset = data.map(tokenize_function, batched=True)
predict_dataset.set_format(type="torch", columns=["input_ids", "attention_mask"])

count = 0
# Predict
with open("adverb_discover.csv", "w+") as fout:
    for example in tqdm(predict_dataset):
        decode = tokenizer.decode(example["input_ids"], skip_special_tokens=True)
        if len(decode) > 160:
            continue
        predictions = trainer.predict([example])
        pred = torch.tensor(predictions.predictions[0]).softmax(dim=-1)
        if pred[-1] < .5:
            continue
        fout.write(f"{decode}\t{pred[-1].item()}\n")
        count += 1
        print(count, f"{decode}\t{pred[-1].item()}\n")

  0%|          | 0/395585 [00:00<?, ?it/s]

1 Although some rejected the European intellectual influence and held to their traditional culture, others <mark>enthusiastically</mark> adopted Western ideas.	0.7988383769989014



2 All of our guests had brought in photos and <mark>willingly</mark> discussed their memories and the stories behind them.	0.9445873498916626



3 He <mark>willingly</mark> lowered himself in his birth, in his ministry, in his death.	0.9777131676673889



4 In calling for this, Stalin advocates a concept that communism until <mark>now</mark> has bitterly opposed.	0.6084185838699341



...

299 Or, do we see <mark>seriously</mark> ill patients as fellow human beings who deserve our love and solutions for their problems?	0.7685819864273071



300 But I am sure 70 there are thousands of <mark>kindly</mark> little deeds waiting to be 71 today, which are being put off" 72 later."	0.8617264628410339



301 We need to understand what we are feeling, how we are becoming overloaded, and what we can do to treat ourselves <mark>kindly</mark>.	0.8216020464897156



302 <mark>Kindly</mark> allow me know so that I
could subscribe.	0.7136069536209106



KeyboardInterrupt: 